In [1]:
%matplotlib inline
import nupack, numpy as np, matplotlib.pyplot as plt
from scipy.integrate import odeint, OdeSolver

In [2]:
# nupack state info

model = nupack.Model(gt = True)

w = nupack.State('GCGUCGCGUCGCUAUGC',dp = '.....((((....))))', kind='full', model=model) # from Brian's paper

w_unpaired = nupack.State('GCGUCGCGUCGCUAUGC', kind='full', model=model) # unpaired for use with rate matrix
# self = sequence, strands = can be list of strands in state, dp = dot parens (default is unpaired),
# kind = move generation algorithm (lazy, full or static, default is lazy), model = energy model to use

print(w.dp())
print(w.sequence())
print(w.pairs.view())
print(w)
#R = w_unpaired.rate_matrix()[2] # rate matrix from unpaired
#print(R)# / (R.shape[0] * R.shape[1]))

.....((((....))))
GCGTCGCGTCGCTATGC
[ 0  1  2  3  4  5 17 16 15 14 10 11 12 13  9  8  7  6 18]
State('GCGTCGCGTCGCTATGC', '.....((((....))))', -1.5)


In [3]:
analysis = nupack.Analysis(model=model)
sequences = ('GCGUCGCGUCGCUAUGC',)
analysis.boltzmann_sample(sequences, 1e4) # generates 1e4 (or set #) of sample structures within the boltzmann distribution
analysis.pair_probability(sequences)
results = analysis.compute() # compute partition function, MFE, PP matrix, etc. for each sequence in boltzmann sample
pairs = results[sequences].pair_probability
samples = [s.dp() for s in results[sequences].boltzmann_sample]  # return dot-parens for each structure in sample

In [4]:
# Run small box algorithm to get macrostates
# print(R_cg)
#macros, R_cg, pf_cg = nupack.kmc.small_box(10 * [w], timeout = .1, tau=10e-6, match=0.7, spread = 400) # 10 is number of trajectories * [starting state]
macros, R_cg, pf_cg = nupack.kmc.small_box(10 * [w], timeout = .07, tau=10e-6, match=0.7, spread = 400) # 10 is number of trajectories * [starting state]

#Final macrostate MFEs:
#MfeInterval
#states: [State('GCGTCGCGTCGCTATGC', '.....((((....))))', -1.5)]
#MfeInterval
#states: [State('GCGTCGCGTCGCTATGC', '(((......))).....', -2.6)]
#MfeInterval
#states: [State('GCGTCGCGTCGCTATGC', '((((.((...)).))))', -4.1)]

In [ ]:
[model.boltz(e - results[sequences].free_energy) for e in (-1.5, -2.6, -4.1)]
# 

In [9]:
R_cg

array([[-4125.09516473,  2114.11127193,  2010.98389281],
       [ 1340.62373993, -2500.18671118,  1159.56297125],
       [   86.48050499,    78.63663109,  -165.11713609]])

In [10]:
pi_cg = pf_cg/pf_cg.sum()
# print(pi_cg @ R_cg)
# print(pi_cg)
for i in range(3):
    for j in range(3):
        (print(R_cg[i,j] / R_cg[j,i], pf_cg[j] / pf_cg[i]))
# print(pf_cg[1] / pf_cg[0])

# pi_cg2 = np.linalg.eig(R_cg.T)[1][:, -1]
# pi_cg2 /= pi_cg2.sum()

1.0 1.0
1.5769609391220283 1.5769609391220283
23.253609503783007 23.25360950378301
0.6341311158643848 0.6341311158643848
1.0 1.0
14.745837342508581 14.745837342508583
0.04300407641391395 0.04300407641391394
0.06781574872776121 0.06781574872776121
1.0 1.0


In [ ]:
# mean match distance between macrostates (test to compare to set match criteria to see if being met)
np.abs(macros[1].pairs.mean() - macros[2].pairs.mean()).sum() / len(w.pairs.view()) / 2

In [ ]:
# structure matrix function where 1's indicate paired bases (1's on diag indicate unpaired)

def structure_matrix_from_state(w):
    pairs = w.pairs.view()
    S = np.zeros((len(pairs), len(pairs)))
    for i, j in enumerate(pairs):
        S[i, j] = 1
    return S

In [ ]:
# random stuff
np.shape(structure_matrix_from_state(w))
len(w.pairs.view())

In [ ]:
# Option 2 for Trajectories

def single_trajectory(w0, max_time):
    w = w0.copy()
    time = 0
    macrostate_at_time = []
    time_log = []
    while time < max_time:
        P = structure_matrix_from_state(w)
        m_distances = [np.abs(P[1:-1,1:-1] - m.pairs.mean()).sum() / (len(w.pairs.view())-2) for m in macros]
        # use macrostates from small box in trajectory simulation to identify and cluster microstates into macro
        macrostate_at_time.append(np.argmin(m_distances))
        time_log.append(time)
        time += w.step()
    return np.array(macrostate_at_time), np.array(time_log)

trajectories = [print(n, end='\r') or single_trajectory(w, 0.004) for n in range(200)] # just prints the number as it goes

In [ ]:
# example: plot a few trajectories going between 0,1,2
for states, times in trajectories[:3]:
    plt.plot(times, states)

In [ ]:
# bin by time and macrostate to produce trajectory plot

from scipy.interpolate import interp1d

time_grid = np.linspace(0, 0.004, 400)
summed = np.zeros((len(macros), len(time_grid)))

for states, times in trajectories:
    for i in range(len(macros)):
        # (states == i) is just a list of 0 or 1 at each step corresponding to if it's in macrostate i
        # I found this is an easy way of doing 0th order interpolation of the discrete timesteps into the grid:
        interp = interp1d(times, states == i, kind=0, fill_value='extrapolate')   
        summed[i] += interp(time_grid)

summed /= len(trajectories)
        
for i, s in enumerate(summed):
    plt.plot(time_grid, s, label='Macrostate %d' % i)
plt.legend()
plt.xlabel('time(s)')
plt.ylabel('probability')
#plt.s

In [ ]:
# Use partition function from macrostates discovered in course-graining to find the course-grained stationary matrix

pi_cg = pf_cg/pf_cg.sum()

# Define basis as PP matrix for each macrostate, have to adjust dimensions to add extra NT to each end

basis = [] # PP for each macrostate
for m in macros:
    add = np.eye(len(w.pairs.view()))
    add[1:-1, 1:-1] = m.pairs.mean()
    basis.append(add)
basis = np.array(basis)
print(basis.shape)

print(R_cg) # course-grained rate-matrix

In [ ]:
# Calculating S_ij from the course-grained rate matrix to compare to analytic

PI_cg = np.tile(pi_cg, (len(macros),1)) # square matrix of stationary values
Z_cg = - (np.linalg.inv(R_cg+PI_cg) - PI_cg)

S_cg = Z_cg / PI_cg

In [ ]:
# access and store the enumerated states, stationary matrix, rate matrix

enumerated_states, pi, R, distances = w_unpaired.rate_matrix(basis=basis) # make sure this w has no pairs
# distances should be a M x N array of pair probability distances for N microstates and M given PP matrices
assignments = np.argmin(distances, axis=1) # macrostate assigned to evert microstate (length N)

# "exact" stationary probability of macrostates found by summing individual probabilities of corresponding microstates
exact_pi = np.array([pi[assignments == m].sum() for m in range(len(macros))]) # sum pi values corresponding to one macrostate to compute exact pi
    
# R = R.toarray() # expands R from sparse matrix to dense array
# wnew = enumerated_states[0]
#enumerated_states_list = np.ndarray.tolist([enumerated_states])
print(enumerated_states[0].pairs.view())
nupack.PairList('......((((....)))).').view()

In [ ]:
# returns index of enumerated state corresponding to starting structure, if starting structure is not unpaired state
target = nupack.PairList('......((((....)))).').view()   # converts dp structure to pair array
target_index = next(i for i in range(len(enumerated_states)) if np.all(enumerated_states[i].pairs.view() == target)) # searches enumerated states for index of state matching "target" or starting structure
print(target_index)

# find indices of macrostates in enumerated states
macros_dp = [nupack.PairList('......((((....)))).').view(), nupack.PairList('.(((......)))......').view(), nupack.PairList('.((((.((...)).)))).').view()]
macros_index = [0,0,0]
for m in range(len(macros_dp)):
    macros_index[m] = next(int(i) for i in range(len(enumerated_states)) if np.all(enumerated_states[i].pairs.view() == macros_dp[m]))

macros_index = np.array(macros_index, dtype = int)
print(macros_index)

#dir(macros[0].pairs)

In [ ]:
# ODE's to be solved for enumerated solution

def differential_eqs(Prob, t, R):
    return Prob @ R

In [ ]:
# start & stop times, initial state for solving ODE's

tspan = np.linspace(0, 4e-3,1000, dtype = float)

Prob_init = np.zeros(len(enumerated_states), dtype = float)
Prob_init[target_index] = 1

In [ ]:
# run solver for Enumerated Solution

Enumerated_Prob = odeint(differential_eqs,t=tspan,y0=Prob_init, args = (R,)) # much faster to run solver with sparse matrix - why?

In [ ]:
Total_Prob = np.array([Enumerated_Prob[:,assignments == m].sum(axis = 1) for m in range(len(macros))]) # sum columns of values corresponding to one macrostate to compute total probability
print(np.shape(Total_Prob))

In [ ]:
# plot results

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(tspan,Total_Prob[0,:], label = 'Macrostate 1')
plt.plot(tspan,Total_Prob[1,:], label = 'Macrostate 2')
plt.plot(tspan,Total_Prob[2,:], label = 'Macrostate 3')
plt.xlabel('time(s)')
plt.ylabel('probability')
plt.axis([0, 0.004, 0, 1])
plt.legend()

In [ ]:
# Plug in reduced rate matrix from small-box algorithm

Coarse_Grain_Prob = odeint(differential_eqs,t=tspan,y0=np.array([1,0,0], dtype = float), args = (R_cg,)) 
np.shape(Coarse_Grain_Prob)

In [ ]:
plt.plot(tspan,Coarse_Grain_Prob[:,0], label = 'Macrostate 1')
plt.plot(tspan,Coarse_Grain_Prob[:,1], label = 'Macrostate 2')
plt.plot(tspan,Coarse_Grain_Prob[:,2], label = 'Macrostate 3')
plt.xlabel('time(s)')
plt.ylabel('probability')
plt.axis([0, 0.001, 0, 1])
plt.legend()

In [ ]:
# find Z from R and PI

PI = np.tile(pi, (len(enumerated_states),1)) # square matrix of stationary values
Z = - (np.linalg.inv(R+PI) - PI)

# use Z to compute hitting time between states, store in hitting time matrix, H

H = np.zeros((len(enumerated_states), len(enumerated_states)))
iter_range = range(len(enumerated_states)-1)

for i in iter_range:
    for j in iter_range:
        if i != j:
            H[i,j] = (Z[j,j] - Z[i,j])/pi[j]
# print(H)

In [ ]:
# Calculate S_{i,j} to get mu(x) for Analytic Solution
S_ij = Z/PI

# S_AA stores S_ij values for just macrostates
S_AA = S_ij[macros_index][:, macros_index]  # pulls values from S_ij from macrostates

# S_xA stores S_ij values of interaction of other states with macrostates
S_xA = S_ij[:, macros_index[:]] 
print(np.shape(S_ij), np.shape(S_AA), np.shape(S_xA))

In [ ]:
S_xA = np.array(S_xA)
S_AA = np.array(S_AA)

In [ ]:
# calculate mu, the commitor function, from the S matrices

mu_x = (((1 - S_xA @ np.linalg.inv(S_AA) @ np.ones((3,3))) / np.linalg.inv(S_AA).sum() + S_xA) @ np.linalg.inv(S_AA))

In [ ]:
# calculate reduced rate matrix for Analytical Solution

reduced_pi = pi @ mu_x

# calculate parameters for finding reduced rate matrix
diag_pi = np.diag(pi)
diag_pi_inv = np.diag(1/reduced_pi) 

# calculate reduced R for analytic solution
R_analytic = np.array(diag_pi_inv @ np.transpose(mu_x) @ diag_pi @ R @ mu_x)
print(R_analytic)

# print(np.linalg.eigvals(R_analytic))
# print(reduced_pi @ R_analytic)
# print(R_analytic.sum(1))

In [ ]:
# pi_analytic = np.tile(reduced_pi, (3,1)) # square matrix of stationary values
# define a function to calculate Z
def Z_from_R(pi, R):
    return np.linalg.inv(-R + pi[None]) - pi[None]    # pi[None] creates square matrix of pi values

Z_analytic = Z_from_R(reduced_pi, R_analytic)
S_analytic = Z_analytic / reduced_pi
S_analytic

In [ ]:
# CHECK: there's something wrong with partition function from small box, leads to wrong pi_cg
# pi_cg2 is correct, need to fix this
# Z_cg = Z_from_R(pi_cg2, R_cg)
# print(pi_cg2 @ R_cg)
# S_cg = Z_cg / pi_cg2
# np.linalg.norm(S_cg - S_analytic) / np.linalg.norm(S_analytic)

# Z_cg = Z_from_R(pi_cg, R_cg)
print(pi_cg @ R_cg)
# S_cg = Z_cg / pi_cg
# np.linalg.norm(S_cg - S_analytic) / np.linalg.norm(S_analytic)

In [ ]:
# look at error in summarizing system within "N macrostates"

eigvals = np.sort(np.linalg.eigvals(Z))[::-1]
plt.plot(1 - (np.cumsum(eigvals) / eigvals.sum())[:10], marker='.')
# plt.yscale('log')

# Thoughts: ~13% error by using only two macrostates, not huge savings in using 3 over 2
# Would this maybe be a feasible method to initially calculate how many feasible macrostates
# should be explored --> determine some of the match/stop criterion for algorithm at start?
# should be doable since R & pi are sparse matrices, so shouldn't cost much to calculate Z, eigvals of Z & # of feasible macrostates

In [ ]:
# Plug in reduced rate matrix from analytic solution
#help(odeint)
Analytic_Prob = odeint(differential_eqs,t=tspan,y0=np.array([1,0,0], dtype = float), args = (R_analytic,)) 
print(pi_cg, reduced_pi, exact_pi)

In [ ]:
R_analytic

In [ ]:
plt.plot(tspan,Analytic_Prob[:,0], label = 'Macrostate 1')
plt.plot(tspan,Analytic_Prob[:,1], label = 'Macrostate 2')
plt.plot(tspan,Analytic_Prob[:,2], label = 'Macrostate 3')
plt.xlabel('time(s)')
plt.ylabel('probability')
plt.axis([0, 0.004, 0, 1])
plt.legend()

In [ ]:
print(np.power(pi_cg, 2), pi_cg)
print(0.04555903*0.04555903)

In [ ]:
print(S_cg, S_AA)

In [ ]:
# Measurements/Metrics for accuracy of course-grained rate matrix

# compare course-grain and actual stationary matrices
pi_error = np.amax(np.abs(pi_cg - exact_pi)) * 2

# compare S values for course-grained and analytic
#S_error = np.sqrt((np.power(S_cg, 2) / S_AA).sum() - 1)
#S_error = np.sqrt(np.abs((np.power(S_cg, 2) / S_AA).sum() - 1))

S_eigvals = np.linalg.eigvals(np.diag(pi) @ S_ij @ np.diag(pi))
denominator = S_eigvals[np.argsort(S_eigvals)[-3:]].sum()

numerator = np.sqrt((reduced_pi @ np.power((np.diag(reduced_pi) @ S_AA @ np.diag(reduced_pi)),2)).sum()) 
Macros_error = numerator / np.power(denominator, 0.5)
#

print(numerator, denominator)

print(pi_error, Macros_error)
#(np.power(S_cg, 2) / S_AA).sum() ,

In [ ]:
# comparing CPU time of small-box vs. enumerated trajectory
import time

small_box_time = []
for n in range(10):
    t_start = time.time()
    nupack.kmc.small_box(1 * [w], timeout = .2, tau=10e-6, match=0.6, spread = 300)
    t_end = time.time()
    small_box_time.append(t_end - t_start)

print('small box time: ', np.mean(small_box_time))




In [ ]:
# dense array needed for np.linalg.eigvals
R_dense = R.toarray()

In [ ]:
# calculate eigenvalues of enumerated R to find tau_2 (relaxation time for running simulation)
enumerated_eigvals = np.linalg.eigvals(R_dense)

# retrieve second to last (first nonzero) negative eigenvalue to calculate tau_2
tau2 = np.abs(1 / enumerated_eigvals[np.argsort(enumerated_eigvals)[-2]])
print(tau2)
np.mean(small_box_time)/tau2

In [ ]:
enumerated_relaxation_time = []
for n in range(10):
    t_start = time.time()
    w.run(time = tau2)
    t_end = time.time()
    enumerated_relaxation_time.append(t_end - t_start)

print('enumerated relaxation time: ', np.mean(enumerated_relaxation_time))

In [ ]:
# calculate pair probability from enumerated -- COMPLETE
# calculate hitting time between 2 example structures via R and Z -- COMPLETE
# calculate the same using the trajectory code -- COMPLETE
# check eigenvalues and stationary state of R -- COMPLETE
# np.linalg.expm(R * t)
# write some function to do macrostate assignment from a single state and macrostate pair probabiltiies -- COMPLETE
# run some checks/error estimates to see how accurate algorithm estimates are

In [ ]:
int(enumerated_states[0].pairs ^ macros[0].mfe.states[0].pairs) # hamming distance between state & macrostate

In [ ]:
print(w.pairs ^ w2.pairs) # hamming distance (# different base pairs * 2)
w.pairs.view()
# macrostate_of_w = np.argmin([w.pairs ^ macrostate.pairs for macrostate in macrostates])

# macrostate_of_w = np.argmin([(structure_matrix(w.pairs) - macrostate.pair_probability).abs().sum() for macrostate in macrostates])

In [ ]:
# features of Nupack Analysis?

result = nupack.Analysis().pair_probability(w.sequence()).value()
pf = np.exp(result.log_partition_function)
P = result.pair_probability
Pnew = nupack.kmc.PairIntegrator()
Pnew

In [ ]:
np.abs(P - structure_matrix_from_state(w)[1:-1, 1:-1]).sum() / len(P)

In [ ]:
# Option 1 for Trajectories

pairs = nupack.kmc.PairIntegrator()
def print_state(w, t):
    print(w, t)
w2, timer = w.run(time=0.01, observers=[pairs, print_state])
print(timer.time, pairs.mean())

In [ ]:
stop = nupack.kmc.FirstCollision(w.pairs)
w.step()

trajectory = []
def append(state, timestep):
    trajectory.append((w.pairs.copy(), timestep))
    
w.run(stop=stop, observers=[nupack.kmc.PairIntegrator(), append])#[-1].average()

In [ ]:
# calculate enumerated pair probability matrix

pair_probability = np.zeros((len(w.pairs.view()), len(w.pairs.view())))
for k in range(0,len(enumerated_states)):
    wnew = enumerated_states[k]
    pair_probability += state_probability[k]*structure_matrix_from_state(wnew)
#pair_probability

In [ ]:
# run trajectories from unpaired state (start with 200) to find pair probability matrix
starting_pairs = w.pairs.copy()
trajectory_track = []
time_log = []
PP = []
for n in range(0,199):
    t = 0
    while t < 1e-3:
        stop_condition = nupack.kmc.FirstCollision(starting_pairs)
        w2, updated_stop_condition, timer, PPnew = w.run(stop = stop_condition, observers=[nupack.kmc.Timer(),nupack.kmc.PairIntegrator()])
        starting_pairs = w2.pairs.copy()
        trajectory_track.append(w2)
        time_log.append(timer)
        PP.append(PPnew)

In [ ]:
# stop_condition = nupack.kmc.Timer(time=0.2) # or steps=
#stop_condition = nupack.kmc.StopWatch(time=0.2)
stop_condition = nupack.kmc.FirstCollision(starting_pairs)

w2, updated_stop_condition, timer, PP2 = w.run(stop=stop_condition, observers=[nupack.kmc.Timer(),nupack.kmc.PairIntegrator()])
print(timer.time)

# print(pairs.average())


# nupack.kmc.small_box(...)

In [ ]:
print(w)
starting_pairs = w.pairs.copy()
timestep = w.step()
print(w, timestep)